In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\Student-predictor\\student app\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\Student-predictor\\student app'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_df: Path

In [6]:
from studentApp.constants import *
from studentApp.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            train_df=Path(config.train_df),
          
        )

        return  data_transformation_config


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [11]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.train_df = None
        self.train = None
    

    def transformation_data(self):

        train_df = pd.read_csv(self.config.train_df)

        X = train_df.drop(columns=['math_score'],axis=1)
        y = train_df['math_score']

        # Create Column Transformer with 3 types of transformers
        num_features = X.select_dtypes(exclude="object").columns
        cat_features = X.select_dtypes(include="object").columns

        numeric_transformer = StandardScaler()
        oh_transformer = OneHotEncoder()

        preprocessor = ColumnTransformer(
            [
                ("OneHotEncoder", oh_transformer, cat_features),
                ("StandardScaler", numeric_transformer, num_features),        
            ]
        )
        X = preprocessor.fit_transform(X)


        print("Data transformation complete")


In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transformation_data()
   
except Exception as e:
    raise e

[2023-07-17 21:43:02,736: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-17 21:43:02,744: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-17 21:43:02,744: INFO: common: created directory at: artifacts]
[2023-07-17 21:43:02,753: INFO: common: created directory at: artifacts/data_ingestion]
Data transformation complete
